# Infer ***cell mask*** from a cell membrane marker - part 2️⃣

--------------

## OBJECTIVE:  
### ✅ Infer sub-cellular component #2: ***cellmask*** in order to understand interactome 

The cell mask will be infered from the cell membrane marker. The fluorescently tagged membrane will be labeled and the inverse of the area will be used as the cellmask.

Dependencies:

Proper cell area inference is necessary for understanding the organelle interacome of the cell.



# IMPORTS

In [2]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice, vesselness3D
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.core.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray, read_czi_image)
from infer_subc_2d.core.img import *

%load_ext autoreload
%autoreload 2

test_img_n = 5

# SETUP

In [3]:
# this will be the example for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Documents\Python Scripts\Infer-subc-2D" 

# linearly unmixed ".czi" files are here
data_path = data_root_path / "8 channel iN day21 deconv"
im_type = ".tiff"

# get the list of all files in "raw"
img_file_list = list_image_files(data_path,im_type)
# test_img_name = img_file_list[test_img_n]
# test_img_name

img_file_list

['C:\\Users\\Shannon\\Documents\\Python Scripts\\Infer-subc-2D\\8 channel iN day21 deconv\\11292022_ZS_BR1_iNDay21_N01_Linear unmixing_0_cmle.ome.tiff',
 'C:\\Users\\Shannon\\Documents\\Python Scripts\\Infer-subc-2D\\8 channel iN day21 deconv\\11292022_ZS_BR1_iNDay21_N02_Linear unmixing_0_cmle.ome.tiff',
 'C:\\Users\\Shannon\\Documents\\Python Scripts\\Infer-subc-2D\\8 channel iN day21 deconv\\11292022_ZS_BR1_iNDay21_N03_Linear unmixing_0_cmle.ome.tiff',
 'C:\\Users\\Shannon\\Documents\\Python Scripts\\Infer-subc-2D\\8 channel iN day21 deconv\\11292022_ZS_BR1_iNDay21_N04_Linear unmixing_0_cmle.ome.tiff',
 'C:\\Users\\Shannon\\Documents\\Python Scripts\\Infer-subc-2D\\8 channel iN day21 deconv\\11292022_ZS_BR1_iNDay21_N05_Linear unmixing_0_cmle.ome.tiff']

In [4]:
#select one image
test_img = img_file_list[0]

# isolate image as an ndarray and metadata as a dictionary
img_data, meta_dict = read_czi_image(test_img)

# # get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
huygens_meta = meta_dict['metadata']['raw_image_metadata']
file_name = meta_dict['file_name'].split('\\')[-1][:-9]

c:\Users\Shannon\Anaconda3\envs\infer-subc-2D\lib\site-packages\ome_types\_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [5]:
viewer = napari.Viewer()
viewer.add_image(img_data, scale=scale)

29-Mar-23 16:58:27 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1090x687+1055-385 (frame: 1108x734+1046-423) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1020x921+1053-392 (frame: 1038x968+1044-430) margins: 9, 38, 9, 9 minimum size: 612x594 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=630,641 maxtrack=0,0)
29-Mar-23 16:58:27 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1664x1032+1053-392 (frame: 1686x1088+1042-437) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY2". Resulting geometry: 1996x1239+1055-385 (frame: 2018x1295+1044-430) margins: 11, 45, 11, 11 minimum size: 612x593 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=1246,1242 maxtrack=0,0)


<Image layer 'img_data' at 0x169c7c82f70>

----------------------------------------
# infer ***cellmask***


### Summary of steps:
INPUT
* Isolate the cell membrane marker - channel 7

PRE-PROCESSING
* Rescale image and smooth (if necessary).

CORE PROCESSING 
* Segment the fluorescent cell membrane using vesselness3D() filter from aicssegmentation.core.vessel.py.

POST PROCESSING
* Filter out small objects and fill small holes (as necessary).
* Invert the mask to get the cell area.

OUTPUT
* inferred ***cellmask*** object

### INPUT

In [6]:
###################
# INPUT
###################
raw_cellmem = img_data[7]

In [7]:
viewer.add_image(raw_cellmem, scale=scale)

<Image layer 'raw_cellmem' at 0x169b5598df0>

## PREPROCESSING

In [8]:
###################
# PREPROCESSING
###################
median_sz = 0
gauss_sig = 0
struct_img_smooth = scale_and_smooth(raw_cellmem, median_sz=median_sz, gauss_sig=gauss_sig)

viewer.add_image(struct_img_smooth, scale=scale)

<Image layer 'struct_img_smooth' at 0x169b5598880>

In [9]:
###################
# CORE
###################
siglist = [2.5]
tau = 0.5
cutoff = 0.0001
bw = vesselness3D(struct_img_smooth, sigmas=siglist, tau=tau, cutoff=cutoff)

viewer.add_image(bw, scale=scale, opacity=0.25, colormap='green')

<Image layer 'bw' at 0x169b355bcd0>

In [13]:
method = 'ave'
MOcutoff = 50
local_adjust = 0.05
mask_test = MO(struct_img_smooth, global_thresh_method=method, object_minArea=MOcutoff, extra_criteria=True, local_adjust=local_adjust)

In [14]:
viewer.add_image(mask_test, scale=scale, opacity=0.25, colormap='bop orange')

<Image layer 'mask_test' at 0x169a6ac07f0>

SR stopped working on this because the membrane marker after deconvolution doesn't show up well enough to create a clear segmentation.